In [98]:
import pandas as pd
import json
import fasttext

# Loading merged data from channels

In [72]:
df_channels = pd.read_csv('/Users/katerynaburovova/PycharmProjects/dehumanization/data/merged_dataset/df_channels.csv', header=None)

In [73]:
df_channels.columns = df_channels.iloc[0]
df_channels = df_channels[1:]

In [74]:
df_channels = (df_channels
               .drop(columns=['Unnamed: 0'], axis=1)
               .reset_index(drop=True))

In [75]:
df_channels

,id,date,views,reactions,to_id,fwd_from,message,type,duration,frw_from_name,channel_name
0,2314,2022-11-21 09:03:37+00:00,546.0,MessageReactions(results=[ReactionCount(reacti...,PeerChannel(channel_id=1182607945),"MessageFwdHeader(date=datetime.datetime(2022, ...","Главный редактор ИА Regnum, писатель, журналис...",text,NaN,Специально для RT,dolgarevaanna
1,115121,2022-11-21 09:06:32+00:00,NaN,NaN,PeerChannel(channel_id=1591547950),NaN,"Прям много видел поляков?Вы хоть сами верите,в...",text,NaN,NaN,dolgarevaanna
2,115120,2022-11-21 09:06:29+00:00,NaN,NaN,PeerChannel(channel_id=1591547950),NaN,ЗА НАШИХ ПАЦАНОВ С#КИ!\n\nСегодня утром Украин...,text,NaN,NaN,dolgarevaanna
3,2313,2022-11-20 17:36:23+00:00,14524.0,MessageReactions(results=[ReactionCount(reacti...,PeerChannel(channel_id=1182607945),NaN,"Киев. Расследует. Киев расследует. Киев, блин,...",text,NaN,NaN,dolgarevaanna
4,115119,2022-11-21 09:05:38+00:00,NaN,NaN,PeerChannel(channel_id=1591547950),NaN,"Пригожин, к слову, непрозрачно дал намекнуть, ...",text,NaN,NaN,dolgarevaanna
...,...,...,...,...,...,...,...,...,...,...,...
182,24114,2022-11-21 06:03:13+00:00,285147.0,NaN,PeerChannel(channel_id=1497011710),NaN,"Доброе утро, друзья мои. Начинаем с политики: ...",text,NaN,NaN,bloodysx
183,24113,2022-11-20 16:02:48+00:00,473577.0,NaN,PeerChannel(channel_id=1497011710),NaN,NaN,photo,NaN,NaN,bloodysx
184,24112,2022-11-20 16:02:48+00:00,471682.0,NaN,PeerChannel(channel_id=1497011710),NaN,Хорошие новости! \n\nМы вынудили столичную пол...,photo,NaN,NaN,bloodysx
185,24111,2022-11-20 14:02:50+00:00,520084.0,NaN,PeerChannel(channel_id=1497011710),NaN,"Кажется, IKEA пользовалась трудом политзаключе...",text,NaN,NaN,bloodysx


FWD from will be used for network analysis

In [78]:
def extract_id(text):
    if text is None:
        return ""
    else:
        pos = text.find('channel_id=')+len('channel_id=')
        result = text[pos:(pos+25)].split('),')[0]
        return result

In [80]:
df_channels['frw_from_id_clean'] = df_channels['fwd_from'].apply(lambda x: extract_id(str(x)))

In [82]:
indices = (df_channels['frw_from_id_clean']
              .value_counts()
              .reset_index(name="count")[1:]['index']
              .to_list())

['1795993534',
 '1686844692',
 '1721909813',
 '1756041380',
 '1036362176',
 '1443086002',
 '1069772626',
 '1193426706',
 '1650753773',
 '1210987817',
 '1216119496']

In [85]:
titles = (df_channels['frw_from_name']
              .value_counts()
              .reset_index(name="count")[1:]['index']
              .to_list())
titles

['Специально для RT',
 '«КПД» (Колобродов, Прилепин, Демидов)',
 'Иди и смотри',
 'RT на русском',
 'BRIEF MOSCOW',
 'Пул N3',
 '| V | ОД "Свободный Донбасс"',
 'Гражданин ГуZzz',
 'Неофициальный Безсонов "Z"',
 'В Москве солнечно']

In [93]:
dict_id = {"ids": indices}
json_object = json.dumps(dict_id, indent=4)
with open("/Users/katerynaburovova/PycharmProjects/dehumanization/data/ids_channels_list.json", "w") as outfile:
    outfile.write(json_object)

In [94]:
dict_ttl = {"titles": titles}
json_object = json.dumps(dict_ttl, indent=4)
with open("/Users/katerynaburovova/PycharmProjects/dehumanization/data/titles_channels_list.json", "w") as outfile:
    outfile.write(json_object)

# Exploring the data

In [99]:
class LanguageIdentification:

    def __init__(self):
        pretrained_lang_model = "/Users/katerynaburovova/PycharmProjects/comp_soc_sci_projects/fasttext/lid.176.bin"
        self.model = fasttext.load_model(pretrained_lang_model)

    def predict_lang(self, text, label_only=True):
        predictions = self.model.predict(text, k=1)
        if label_only:
            return predictions[0][0][-2:]
        return predictions

In [100]:
lang_identifier = LanguageIdentification()
df_channels['message'] = df_channels['message'].apply(lambda x: str(x).replace('\n', ' '))
df_channels['lang'] = df_channels['message'].apply(lambda x: lang_identifier.predict_lang(x))

In [101]:
df_channels['lang'].value_counts()

ru    171
ht      9
uk      3
ja      1
pl      1
sv      1
it      1
Name: lang, dtype: int64

In [103]:
df_channels[df_channels['lang']=='uk']

,id,date,views,reactions,to_id,fwd_from,message,type,duration,frw_from_name,channel_name,frw_from_id_clean,lang
51,114890,2022-11-20 09:24:40+00:00,NaN,NaN,PeerChannel(channel_id=1591547950),NaN,Ура!),text,NaN,NaN,dolgarevaanna,,uk
111,2317643,2022-11-21 09:03:05+00:00,NaN,NaN,PeerChannel(channel_id=1234644047),NaN,Це пiзда,text,NaN,NaN,rosich_ru,,uk
158,2317605,2022-11-21 08:58:52+00:00,NaN,NaN,PeerChannel(channel_id=1234644047),NaN,А врач- точно врач?),text,NaN,NaN,rosich_ru,,uk


# Set up for the influence maximization problem

Influence maximization is the problem of finding a small set of most influential nodes in a social network so that their aggregated influence in the network is maximized (maximizing the information diffusion). The most widely used diffusion models include classical epidemic outbreak model, independent cascade model, linear threshold model, Weight Cascade(wc), trigger and rumor-spreading model (with independent cascade model, linear threshold model being more popular approaches).

We could either use the threshold for channel to qualify, or use some kind of connectivity (centrality?) measure to estimate the influence and include top-X channel out of all the channels we've been able to identify.

We collect the info on channels referenced by the ones we parse (id, title, name handle) to use for building the graph. We also use views as a proxy for current audience estimate.

In [105]:
df_channels.head(5)

,id,date,views,reactions,to_id,fwd_from,message,type,duration,frw_from_name,channel_name,frw_from_id_clean,lang
0,2314,2022-11-21 09:03:37+00:00,546.0,MessageReactions(results=[ReactionCount(reacti...,PeerChannel(channel_id=1182607945),"MessageFwdHeader(date=datetime.datetime(2022, ...","Главный редактор ИА Regnum, писатель, журналис...",text,NaN,Специально для RT,dolgarevaanna,1686844692,ru
1,115121,2022-11-21 09:06:32+00:00,NaN,NaN,PeerChannel(channel_id=1591547950),NaN,"Прям много видел поляков?Вы хоть сами верите,в...",text,NaN,NaN,dolgarevaanna,,ru
2,115120,2022-11-21 09:06:29+00:00,NaN,NaN,PeerChannel(channel_id=1591547950),NaN,ЗА НАШИХ ПАЦАНОВ С#КИ! Сегодня утром Украинск...,text,NaN,NaN,dolgarevaanna,,ru
3,2313,2022-11-20 17:36:23+00:00,14524.0,MessageReactions(results=[ReactionCount(reacti...,PeerChannel(channel_id=1182607945),NaN,"Киев. Расследует. Киев расследует. Киев, блин,...",text,NaN,NaN,dolgarevaanna,,ru
4,115119,2022-11-21 09:05:38+00:00,NaN,NaN,PeerChannel(channel_id=1591547950),NaN,"Пригожин, к слову, непрозрачно дал намекнуть, ...",text,NaN,NaN,dolgarevaanna,,ru
